In [10]:
from scipy.interpolate import interp1d
from specutils import Spectrum1D
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from specutils.manipulation import FluxConservingResampler
from pybaselines import whittaker
import numpy as np
import matplotlib.pyplot as plt
import pickle
import astropy.units as u
from astropy.io import fits
import scipy.constants as cst
from astropy.stats import gaussian_fwhm_to_sigma
from astropy.cosmology import Planck15 as cosmo
from functools import partial
import pandas as pd
from matplotlib.pyplot import cm
import matplotlib.lines as mlines
from astropy.stats import gaussian_fwhm_to_sigma
from astropy.modeling.models import Gaussian1D
from scipy.interpolate import interp1d
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
import matplotlib.markers as mmarkers

line_list_fancy = ["CO({}-{})".format(J_up, J_up - 1) for J_up in range(1, 9)]
rest_freq_list = [115.27120180 * J_up for J_up in range(1, 9)]

def give_map_spatial_freq(res, ny, nx):
    Ys, Xs = np.unravel_index(np.arange(nx*ny),(ny,nx))
    N = np.zeros((ny, nx))
    M = np.zeros((ny, nx))
    for  xs, ys, in zip(Xs, Ys):
        N[ys,xs] = int(xs)
        M[ys,xs] = int(ys)
    w_sup = np.where(M>ny/2)
    M[w_sup] = ny - M[w_sup]
    w_sup = np.where(N>nx/2)
    N[w_sup] = nx - N[w_sup]
    Kx = N/nx/res
    Ky = M/ny/res
    k_map = (Kx**2 + Ky**2)**(1/2)
    return k_map

def give_map_spatial_freq_one_axis(res, ny):
    Ys = np.arange(ny)
    M = np.zeros(ny)
    for ys, in zip(Ys):
        M[ys] = int(ys)
    w_sup = np.where(M>ny/2)
    M[w_sup] = ny - M[w_sup]
    Ky = M/ny/res
    k_map = (Ky**2)**(1/2)
    return k_map

In [9]:
#Spatial deep
FWHM_HF = 30 #arcsec
FWHM_LF = 35
dnu = 1.5
nu_range_HF = (195,310)
nu_range_LF = (130,195)
D = 11
field_size = 1.5*u.deg**2  

#in a first approximation, what SNR do we have if we want to measure the cross spec of each line at z in [z-dz, z+dz] ? 
dz = 0.1
for dz


for line, rest_freq in zip(line_list_fancy, rest_freq_list):    
        z = rest_freq / nu_list -1 

for FWHM, (nu_min, nu_max) in zip((FWHM_HF, FWHM_LF), (nu_range_HF, nu_range_LF)):

    res_angular = 2*FWHM
    ragrid= np.arange(0,np.sqrt(field_size),(res_angular*u.arcsec).to(u.deg).value)
    decgrid= np.arange(0,np.sqrt(field_size),(res_angular*u.arcsec).to(u.deg).value)

    ra_center = np.mean(ragrid)
    dec_center = np.mean(decgrid)
    delta_ra = np.max(ragrid) - np.min(ragrid)
    delta_dec = np.max(decgrid) - np.min(decgrid)
    nu_list = np.arange(nu_min, nu_max, dnu)


        Dc_center = cosmo.comoving_distance(z).value

        #all the coordinates will be in comoving units
        size_x = Dc_center * delta_ra * (np.pi/180) * np.cos(np.pi/180*dec_center)
        size_y = Dc_center * delta_dec * (np.pi/180) 

        #compute the coordinates in the cube
        ys = Dc_center * ( ragrid[:, np.newaxis]  - ra_center)  * (np.pi/180) * np.cos(np.pi/180*ragrid[:, np.newaxis])
        xs = Dc_center * ( decgrid[:, np.newaxis] - dec_center) * (np.pi/180)

        #box_size = np.min([size_x, size_y])
        #bins = np.linspace(-box_size/2, box_size/2, num = Nvox + 1)

        grid=np.array(np.meshgrid(ys,xs))
        cube = np.zeros((len(zcii), len(ragrid), len(decgrid), 3))
        for i,c in zip(range(len(zcii)), cm.plasma(np.linspace(0,1,len(zcii)))):
            for j in range(len(ragrid)):
                for k in range(len(decgrid)):
                    cube[i,j,k,0] = Dc_center[i]
                    cube[i,j,k,1] = ys[j, i]
                    cube[i,j,k,2] = xs[k, i]

        dc_center = cube[int(cube.shape[0]/2),int(cube.shape[1]/2),int(cube.shape[2]/2),0]
        centered_cube = cube.copy()
        centered_cube[:,:,:,0] -= dc_center

        BS = 10; plt.rc('font', size=BS); plt.rc('axes', titlesize=BS); plt.rc('axes', labelsize=BS); mk = 3; lw=1
        fig, ax = plt.subplots(figsize=(4, 3.3), dpi = 200)
        patchs = []

        for i, (imin, imax, c) in enumerate(zip(z_bins_index, z_binswidth_index, ('g', 'r',  'b', 'magenta'))):
    
            patch = mpatches.Patch(color=c, label=f'z={np.round(zcii_tim[iz],2)}' ); patchs.append(patch);
            subcube = cube[int(iz-idz):int(iz+idz+1),:,:,:]
            Dc_center = cube[iz,0,0,0]
            subcube[:,:,:,0] -= Dc_center
                    
            kmaps_list = []
            kmaps_nyquist = []
            kperp_min_list = []
            kperp_max_list = []
                    
            for zi in range(len(subcube[:,0,0,1])):
                res = subcube[zii, 1, 0,1]-subcube[zii, 0, 0, 1]
                if(zii==0): res_perp_0 = res
                kmap = 2*np.pi*give_map_spatial_freq(res, r_cylindrical.shape[1], r_cylindrical.shape[0])
                kmaps_list.append(kmap)

            Z = np.abs(subcube[:,0,0,0])
            lenght = Z[0]+Z[-1]
            res = Z[0]-Z[1]
            kz = 2*np.pi*give_map_spatial_freq_one_axis(res, len(z))
            kz_arr = np.zeros(np.asarray(kmaps_list).shape)
            kz_arr[:,:,:] = kz[:,np.newaxis, np.newaxis]
            k = np.sqrt(np.asarray(kmaps_list)**2+kz[:,np.newaxis, np.newaxis]**2)/u.Mpc

            #Setting the binning of the k map
            k_binsnp = np.logspace(np.log10(k[k>0].min().value), np.log10(k.max().value), 20)
            nmodes, kbins = np.histogram(k, bins = k_binsnp/u.Mpc)
            kmean, kbins = np.histogram(k, bins = k_binsnp/u.Mpc, weights=k)
            kmean /= nmodes
            ax.stairs(nmodes,k_binsnp, color=c)

            Vtot = 4/3*np.pi*(cosmo.comoving_distance(zcii_tim[int(iz+idz)])**3-cosmo.comoving_distance(zcii_tim[int(iz-idz)])**3)
            Vsurvey =  (field_size*u.deg**2).to(u.sr) * Vtot / (4*np.pi)
            Nmodes = 2 * np.pi * kmean**2 * np.diff(k_binsnp) * Vsurvey / (2*np.pi)**3
            ax.plot(kmean, Nmodes, color=c, ls='--')
            #ax.plot(kmean, Nmodes/4e3, color=c, ls=':')


        patch = mlines.Line2D([], [], color='k', linestyle="solid",label='count from fft' ); patchs.append(patch);
        patch = mlines.Line2D([], [], color='k', linestyle="--",label='$\\rm 2 \\pi k^2 \\Delta k *V_{survey} / (2\\pi)^3$' ); patchs.append(patch);
        #patch = mlines.Line2D([], [], color='k', linestyle=":",label='$\\rm 2 \\pi k^2 \\Delta k *V_{survey} / (2\\pi)^3/4e3$' ); patchs.append(patch);

        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.set_ylabel('$\\rm N_{modes}(k)$')
        ax.legend(loc='upper left',handles = patchs, fontsize=8)
        ax.set_xlabel(r'$\rm ||k||$ [$\rm Mpc^{-1}$]')
        plt.rcParams.update({'xtick.direction':'in'})
        plt.rcParams.update({'ytick.direction':'in'})
        plt.rcParams.update({'xtick.top':True})
        plt.rcParams.update({'ytick.right':True})
        plt.rcParams.update({'legend.frameon':False})
        
